# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howard springs,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
1,1,isafjordur,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
2,2,namsos,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
3,3,mahina,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
4,4,san patricio,-12.497,131.047,26.95,67,46,3.27,AU,1693475429


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.5,
    color="City",
    hover_cols=["Lng", "Lat", "City", "Humidity"],
)
# Display the map
city_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_cities_df = city_data_df.query("Humidity >= 30").query("Cloudiness <= 70")
# Drop any rows with null values
Ideal_cities_df.dropna(axis=0, inplace=True)

# Display sample data
Ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,howard springs,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
1,1,isafjordur,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
2,2,namsos,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
3,3,mahina,-12.497,131.047,26.95,67,46,3.27,AU,1693475429
4,4,san patricio,-12.497,131.047,26.95,67,46,3.27,AU,1693475429


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(
    {
        "City": Ideal_cities_df["City"],
        "Country": Ideal_cities_df["Country"],
        "Lat": Ideal_cities_df["Lat"],
        "Lng": Ideal_cities_df["Lng"],
        "Humidity": Ideal_cities_df["Humidity"],
    }
)
hotel_df = hotel_df.reset_index(drop=True)
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,howard springs,AU,-12.497,131.047,67,
1,isafjordur,AU,-12.497,131.047,67,
2,namsos,AU,-12.497,131.047,67,
3,mahina,AU,-12.497,131.047,67,
4,san patricio,AU,-12.497,131.047,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
radius = 10000
params = {"apiKey": geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df["Lat"][index]
    longitude = hotel_df["Lng"][index]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] = f"accommodation.hotel"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"][
            "name"
        ]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(
        f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}"
    )

# Display sample data
hotel_df

Starting hotel search
howard springs - nearest hotel: No hotel found
isafjordur - nearest hotel: No hotel found
namsos - nearest hotel: No hotel found
mahina - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
brookings - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: No hotel found
kiunga - nearest hotel: No hotel found
mahonda - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
whitehorse - nearest hotel: No hotel found
taizhou - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
taiohae - nearest hotel: No hotel found
colon - nearest hotel: No hotel found
naze - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
saint-pierre - nearest hotel: No hot

,City,Country,Lat,Lng,Humidity,Hotel Name
0,howard springs,AU,-12.4970,131.0470,67,No hotel found
1,isafjordur,AU,-12.4970,131.0470,67,No hotel found
2,namsos,AU,-12.4970,131.0470,67,No hotel found
3,mahina,AU,-12.4970,131.0470,67,No hotel found
4,san patricio,AU,-12.4970,131.0470,67,No hotel found
...,...,...,...,...,...,...
195,san antonio del estrecho,BR,-20.3297,-40.2925,79,Hotel Prainha
196,chiili,BR,-20.3297,-40.2925,79,Hotel Prainha
197,bucerias,BR,-20.3297,-40.2925,79,Hotel Prainha
198,inyonga,BR,-20.3297,-40.2925,79,Hotel Prainha


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=1.5,
    color="City",
    hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"],
)

# Display the map
hotel_data_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)